In [1]:
import numpy as np
import csv
import time
import math
import pandas as pd
from numba import cuda, float32, float64
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
from numba import cuda
random.seed(1)

In [2]:
def readData(file):
    file=open(file, 'r')
    reader=csv.reader(file)
    data= []
    for line in reader:
        data.append(line)
    data=np.array(data)
    return data

In [3]:
#Computes wT*x[i] ( the predicted value for datum)
def H(x,w,i):
    sum=0
    for j in range(len(x[0])):
        sum+=x[i][j]*w[j]
    return sum 

In [4]:
def forward_prop(x, weights):
    weights2=weights[n_inputs*n_hidden:]
    z3=0
    #Pass data from the initial layer to the hidden neurons
    for j in range(0,n_hidden-1):
        weights1=weights[n_inputs*j:n_inputs*(j+1)]
        result=0
        for k in range(0,n_inputs):
            result+=(x[k]*weights1[k])
            #result+=0.1
        #Activation function ReLu
        #result=max(0, result)
        #We can add what each hidden neuron contributes to the output layer
        z3+=result*weights2[j]
    #We add the bias
    z3+=weights2[n_hidden-1]
    return z3

In [5]:
#Function for calculating the error
def forward_propagation_Error(x,y,w):
    E=0
    for i in range(0, len(x)):
        wx=0
        wx=forward_prop(x[i], w)
        E+=((wx-y[i][0])**2)
    E=E/len(x)
    return E

In [6]:
class Particle:
    def __init__(self, position, initial_fitness):
         # particles position
        self.particle_position = position
        #initial value of the particle (infinity or minus infinity,
        #depending on whether we want to maximize or minimize)
        self.fitness_particle_position = initial_fitness
        # best position of the particle
        self.local_best_particle_position = []  
        #best initial value of the particle (infinity or minus infinity, 
        #depending on whether we want to maximize or minimize)
        self.fitness_local_best_particle_position = initial_fitness  
        # particle's velocity
        self.particle_velocity = []  
        for i in range(0,num_dimensions):
            #we generate the initial velocity randomly
            self.particle_velocity.append(random.uniform(-vMax, vMax))  
 
    def evaluate(self, x, y, mm):
        self.fitness_particle_position = forward_propagation_Error(x, y, self.particle_position)
        if mm == -1:
            if self.fitness_particle_position < self.fitness_local_best_particle_position:
                # We update the best local position
                self.local_best_particle_position = self.particle_position  
                # We update the best local value 
                self.fitness_local_best_particle_position = self.fitness_particle_position          
        if mm == 1:
            if self.fitness_particle_position > self.fitness_local_best_particle_position:
                 # We update the best local position
                self.local_best_particle_position = self.particle_position  
                # We update the best local value 
                self.fitness_local_best_particle_position = self.fitness_particle_position   
    def update_velocity(self, global_best_particle_position, w, Vmax, c1=2.8, c2=1.3):
        for i in range(0,num_dimensions):
            r1 = random.random()
            r2 = random.random()
            #We calculate the new velocity
            self.particle_velocity[i] = w * self.particle_velocity[i] + c1 * r1 * (self.local_best_particle_position[i] - self.particle_position[i]) + c2 * r2 * (global_best_particle_position[i] - self.particle_position[i])
            
            #We limit the maxmum velocity
            if(self.particle_velocity[i]>Vmax):
                self.particle_velocity[i]=Vmax
            if(self.particle_velocity[i]<-Vmax):
                self.particle_velocity[i]=-Vmax
 
    def update_position(self, bounds):
        for i in range(0,num_dimensions):
            self.particle_position[i] = self.particle_position[i] + self.particle_velocity[i]
 
            #if it reaches the edges, it stays within, it does not exceed the limits
            if self.particle_position[i] > bounds[1]:
                self.particle_position[i] = bounds[1]
            if self.particle_position[i] < bounds[0]:
                self.particle_position[i] = bounds[0]

In [8]:
#PS0 parameters
num_iters=10
num_particles=46
c1=2.8
c2=1.3
factor=10
Fi=c1+c2
Xi=2/(abs(2-Fi-math.sqrt(abs(Fi**2-4*Fi))))
data=readData('Bias_correction_ucl.csv')
#print('Data prior removing',data)
#We remove the first row of names and the last column
data=data[1:,2:24]
#print('Data after removing',data)
# We preprocess the data
datosPandas=pd.DataFrame(data)
#We refill the missing data with the averagge
datosPandas.replace('?',np.NaN,inplace=True)
imp=SimpleImputer(missing_values=np.NaN)
datos=imp.fit_transform(datosPandas)
#print('Datos after pre-processing',datos)
# Shuffle the data list above
np.random.shuffle(datos)
#Percentage of training (in this sample, 50%) and percentage of testing (25%)
p_train = 0.7
#Number of elements in the training and test datasets
len_train=int((len(datos))*p_train)
datos_train=datos[:len_train,:]
datos_test=datos[(len_train):,:]
#We separate the data "x" from the "y" (the data "y" is in the last column)
num_atrib=int(len(datos[0]))-1
x_train=datos_train[:,:num_atrib]
y_train=datos_train[:,num_atrib]
x_test=datos_test[:,:num_atrib]
y_test=datos_test[:,num_atrib]
print('vector x_train ',x_train)
print('vector y_train ',y_train)

vector x_train  [[3.30000000e+01 2.68000000e+01 4.18325768e+01 ... 5.98324000e+01
  2.68650000e+00 5.28332422e+03]
 [2.72000000e+01 1.78000000e+01 6.27743187e+01 ... 5.09312000e+01
  4.12500000e-01 4.47110059e+03]
 [2.89000000e+01 2.08000000e+01 5.82140617e+01 ... 2.87000000e+01
  6.23300000e-01 4.76898047e+03]
 ...
 [2.59000000e+01 2.44000000e+01 7.02739105e+01 ... 5.09312000e+01
  4.12500000e-01 5.60439941e+03]
 [2.88000000e+01 2.41000000e+01 3.74082756e+01 ... 5.46384000e+01
  1.45700000e-01 4.45902344e+03]
 [2.68000000e+01 2.34000000e+01 4.73865318e+01 ... 5.98324000e+01
  2.68650000e+00 5.25598779e+03]]
vector y_train  [35.2 24.5 29.4 ... 28.5 28.4 27.6]


In [9]:
#Data re-scaling
y_train=np.reshape(y_train, (-1,1))
y_test=np.reshape(y_test, (-1,1))
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
#
scaler_x.fit(x_train)
x_train=scaler_x.transform(x_train)
x_test=scaler_x.transform(x_test)
#
scaler_y.fit(y_train)
y_train=scaler_y.transform(y_train)
y_test=scaler_y.transform(y_test)
print('vector x_train scaled ',x_train)
print('vector y_train scaled',y_train)

vector x_train scaled  [[0.74137931 0.83333333 0.28367234 ... 0.23735354 0.50947891 0.56400165]
 [0.40804598 0.34946237 0.55323478 ... 0.19283975 0.06181467 0.06492858]
 [0.50574713 0.51075269 0.49453507 ... 0.08166429 0.10331319 0.2479617 ]
 ...
 [0.33333333 0.70430108 0.64976964 ... 0.19283975 0.06181467 0.76128719]
 [0.5        0.68817204 0.22672266 ... 0.21137899 0.00929189 0.05750774]
 [0.38505747 0.65053763 0.35516294 ... 0.23735354 0.50947891 0.5472047 ]]
vector y_train scaled [[0.82790698]
 [0.33023256]
 [0.55813953]
 ...
 [0.51627907]
 [0.51162791]
 [0.4744186 ]]


In [10]:
#Neural network parameters (as function of the input)
n_inputs=x_train[0].size
n_hidden=math.floor(len(x_train)/(factor*(n_inputs+1)))+1
#We calculate the number of weights necessary to carry out the computation
num_weights=(n_inputs*n_hidden)+n_hidden
bound0=float(max(y_train))#upper limit
bound1=-bound0 #lower limit
vMax=bound0*0.6
weights=[]
for i in range(0,num_particles):
        for j in range(0, num_weights):
            weights.append(random.uniform(-1, 1))
#print('total number of weights ', len(weights))
#print('Initial weights values', weights )

In [11]:
class Neural_Network:
    # the parameters are the data, the number of neurons of the input layer, 
    # the number of neurons of the hidden layer and 
    # the number of neurons of the output layer
    def __init__(self, x, y, n_inputs, n_hidden, weights, num_weights):
        self.input=x
        self.y=y
        
        self.weights=weights
        self.num_weights=num_weights
        
    def train_PSO(self):
        best_position, fitness=PSO_Neural_Network(self.input, self.y,self.weights, self.num_weights,[bound1,bound0], -1, num_particles, 100, 0.9, 0.1, vMax)
        self.weights=best_position
        return fitness
    
    def predict(self, x_test):
        return forward_prop(x_test, self.weights)    

In [12]:
#PSO parallel
@cuda.jit
#def PSO_GPU(x, y):
def PSO_GPU(x, y, particle_positions, particle_velocity, personal_best_particle_position,fitness_personal_best_particle_position, local_best_particle_position, fitness_local_best_particle_position,pool, c1, c2, Vmax, bound0, bound1):
  
    particle= cuda.blockIdx.x #We found the particle that corresponds to the block where the thread is
    nx=cuda.threadIdx.x #Finds the neurons of the hidden layer that this thread is going to calculate 
    #in this thread. Thread '0' does the rest of the particle calculations
   
    #We initialize the vectors in a shared-memory array (common to the block == to the particle)
    z2=cuda.shared.array(shape=n_hidden, dtype=float32)

    for iters in range(0, num_iters):
        #We take particle's position corresponding to the thread
        position=particle_positions[num_weights*particle: num_weights*(particle+1)]

        #We do forward propagation
        fitness_position=0
        weights2=position[n_inputs*n_hidden:]
        for i in range(0, len(x)):
            j=nx
            while(j<n_hidden):
                #Pass from the init layer to the hidden layer
                hidden_result=0
                weights1=position[n_inputs*j:n_inputs*(j+1)]
                for k in range(0,n_inputs-1):
                    hidden_result+=(x[i][k]*weights1[k])
                hidden_result+=weights1[n_inputs-1] #We add the bias
                #Function ReLu (activation of)
                hidden_result=max(0, hidden_result)
                #We save the value in the vector z2, to add it when the total value 
                #is calculated in the output layer
                z2[j]=hidden_result*weights2[j]
                j=j+threadsperblock
   
            # Thread synchronization
            cuda.syncthreads()
           
            #Thread '0' gathers the results of other threads
            if(nx==0):
                result=0
                for h in range (0, n_hidden):
                    result+=z2[h]
                fitness_position+=float((y[i][0]-result)**2)
            # Thread synchronization
            cuda.syncthreads()
            
        if(nx==0):
            #We update the best personal position
            fitness_position=fitness_position/len(x)
            if fitness_position < fitness_personal_best_particle_position[particle]:
                for i in range(0, num_weights):
                    personal_best_particle_position[(num_weights*particle)+i] = particle_positions[num_weights*particle+i]  # we update the best personal position
                # we update the best personal value
                fitness_personal_best_particle_position[particle] = fitness_position 
       
        # We synchronize the threads
        cuda.syncthreads()
       
        if(nx==0):
            #Best local position updating
            if fitness_personal_best_particle_position[particle]< fitness_local_best_particle_position[particle]:
                for i in range(0, num_weights):
                    local_best_particle_position[num_weights*particle+i] = personal_best_particle_position[(num_weights*particle)+i]
                fitness_local_best_particle_position[particle] = fitness_personal_best_particle_position[particle]
           
            if fitness_personal_best_particle_position[int(math.fmod((num_particles+particle-1), num_particles))] < fitness_local_best_particle_position[particle]:
                for i in range(0, num_weights):
                    local_best_particle_position[num_weights*(particle)+i] = particle_positions[num_weights*(int(math.fmod((num_particles+particle-1), num_particles)))+i]
                fitness_local_best_particle_position[particle] = fitness_personal_best_particle_position[int(math.fmod((num_particles+particle-1), num_particles))]
           
               
            if fitness_personal_best_particle_position[int(math.fmod((num_particles+particle+1), num_particles))] < fitness_local_best_particle_position[particle]:
                for i in range(0, num_weights):
                    local_best_particle_position[num_weights*(particle)+i] = particle_positions[num_weights*(int(math.fmod((num_particles+particle+1), num_particles)))+i]
                fitness_local_best_particle_position[particle] = fitness_personal_best_particle_position[int(math.fmod((num_particles+particle+1), num_particles))]
           
        # Thread synchronization
        cuda.syncthreads()
        
        #We choose numbers from the random numbers repository
        r1=pool[2*num_weights*num_particles+particle+iters]
        r2=pool[2*num_weights*num_particles-particle-iters]
       
        i=nx
        #We update each particle's velocity
        while i<num_weights:
            #We calculate the new velocity
            particle_velocity[num_weights*particle+i] = Xi*( particle_velocity[num_weights*particle+i] + c1 * r1 * (personal_best_particle_position[num_weights*particle+i] - particle_positions[num_weights*particle+i]) + c2 * r2 * (local_best_particle_position[num_weights*particle+i] - particle_positions[num_weights*particle+i]))
            #We limit speed to maximum speed
            if particle_velocity[num_weights*particle+i]>Vmax :
                particle_velocity[num_weights*particle+i]=Vmax
            if particle_velocity[num_weights*particle+i]<(Vmax*(-1)):
                particle_velocity[num_weights*particle+i]=(Vmax*(-1))
            i+=threadsperblock
       
        # Wait until all threads finish computing
        cuda.syncthreads()      
        i=nx
        #We update each particle's position
        while i<num_weights:
            particle_positions[num_weights*particle+i] = particle_positions[num_weights*particle+i] + particle_velocity[num_weights*particle+i]
            #if it reaches and edge, it stays in there, it does not trespasses it
            if particle_positions[num_weights*particle+i] > bound0:
                particle_positions[num_weights*particle+i] = bound0
            if particle_positions[num_weights*particle+i] < bound1:
                particle_positions[num_weights*particle+i] = bound1
            i+=threadsperblock
           
        # Wait until all threads finish computing
        cuda.syncthreads()  

In [13]:
def coresPerSM():
    cc_cores_per_SM_dict = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128
    }
    # the above dictionary should result in a value of "None" if a cc match 
    # is not found.  The dictionary needs to be extended as new devices become
    # available, and currently does not account for all Jetson devices
    device = cuda.get_current_device()
    #my_cc = getattr(device, 'COMPUTE_CAPABILITY')
    my_cc=(6,1)
    cores_per_sm = cc_cores_per_SM_dict.get(my_cc)
    return cores_per_sm
#device = cuda.get_current_device()
threadsperblock =  coresPerSM()
blockspergrid=num_particles
#print('device, num weights, threads per block, blocks, c1, c2, vMax, bound0, bound1 ->',getattr(device, 'MULTIPROCESSOR_COUNT'),num_weights,threadsperblock,blockspergrid,c1, c2, vMax, bound0, bound1)

In [14]:
#We generate one pool of random numbers and initial data
pool=[]
fitness_personal_best_particle_position=[]
fitness_local_best_particle_position=[]
for i in range(0,num_particles):
        for j in range(0, num_weights*3):
            pool.append(random.uniform(bound1, bound0))
        fitness_local_best_particle_position.append(float("inf"))
        fitness_personal_best_particle_position.append(float("inf"))
#We load data to the gpu
x_global_mem=cuda.to_device(np.ascontiguousarray(x_train))
y_global_mem=cuda.to_device(np.ascontiguousarray(y_train))
fitness_personal_best_particle_position_global_mem=cuda.to_device(np.asarray(fitness_personal_best_particle_position))
personal_best_particle_position=cuda.to_device(np.zeros(num_weights*num_particles))
local_best_particle_position=cuda.to_device(np.zeros(num_weights*num_particles))
fitness_local_best_particle_position_global_mem=cuda.to_device(np.ascontiguousarray(fitness_local_best_particle_position))
pool_global_mem=cuda.to_device(np.ascontiguousarray(pool))

#We initiate positions and velocities of particles by taking data from the pool of random numbers
particles_positions=pool_global_mem[0:num_weights*num_particles]
particles_velocity=pool_global_mem[num_weights*num_particles:2*num_weights*num_particles]
print("Data loaded in GPU correctly!")

Data loaded in GPU correctly!


In [15]:
#And now it's time to launch the computation in parallel!!!!
start = time.time()
cuda.profile_start()

PSO_GPU[blockspergrid, threadsperblock](x_global_mem, y_global_mem,particles_positions, particles_velocity, personal_best_particle_position,fitness_personal_best_particle_position_global_mem, local_best_particle_position,fitness_local_best_particle_position_global_mem, pool_global_mem,c1, c2, vMax, bound0, bound1)

local_best_particle_position_host=local_best_particle_position.copy_to_host()
fitness_local_best_particle_position=fitness_local_best_particle_position_global_mem.copy_to_host()
cuda.profile_stop()
end = time.time()
time=end - start
print("Execution time(s): ", time)

Execution time(s):  0.9615638256072998


In [16]:
import time
global_best_particle_position=[]
fitness_global_best_particle_position=float("inf")
for i in range(0, num_particles):
    if(fitness_local_best_particle_position[i]<fitness_global_best_particle_position):
        global_best_particle_position=local_best_particle_position_host[num_weights*i:num_weights*(i+1)]
        fitness_global_best_particle_position=fitness_local_best_particle_position[i]

#We calculate the transfer time between the host (CPU) and the device (GPU), and we remove it from the calculation
start2 = time.time()
local_best_particle_position_host=local_best_particle_position.copy_to_host()
fitness_local_best_particle_position=fitness_local_best_particle_position_global_mem.copy_to_host()
end2 = time.time()
time_corr=end - start -(end2-start2)
#Error calculation
Eout=0
Error_Cero=0
prediction=np.zeros(len(x_test))
for i in range(0, len(x_test)):
        prediction[i]=forward_prop(x_test[i], global_best_particle_position)
        # mean standard error (MSE), for regression problems
        Eout+=float((y_test[i][0]-prediction[i]))**2
Eout=Eout/len(x_test)
#results
print("Number of iterations of PSO =", num_iters)
print("Running time(s): ", time_corr)
print('Ein:', fitness_global_best_particle_position)
print('Eout:', Eout)


Number of iterations of PSO = 10
Running time(s):  0.9615638256072998
Ein: 0.12252526810069354
Eout: 164.83082886755884
